## FEATURE SELECTION - FootySage

En este cuaderno se define un flujo de trabajo para la selección y análisis de características relevantes utilizando los datos de *Open Data* de ***StatsBomb***. 

Este proceso comienza con la integración de información proveniente de diversas fuentes, como partidos, alineaciones y eventos, consolidándola en un dataset estructurado. Cada fila del dataset representa un partido, enriquecido con métricas calculadas que reflejan aspectos clave del rendimiento, como estadísticas agregadas de pases, disparos, recuperaciones, faltas, entre otros.

A partir de este dataset, se entrenarán modelos básicos de Machine Learning, como Árboles de Decisión (Decision Tree), Regresión Logística (Logistic Regression) y K-Nearest Neighbors (KNN). El objetivo es evaluar su capacidad de clasificación y obtener conclusiones sobre la influencia de las características seleccionadas. Este enfoque inicial proporcionará una base sólida para ampliar el análisis hacia conjuntos de datos más extensos e implementar modelos más avanzados en futuras fases del proyecto.

### Estudio de una competición en una temporada (Competición regular)

#### Carga de datos y preprocesamiento

En primer lugar seleccionamos la competición y la temporada de esa competición que queremos estudiar.

In [1]:
from src.fetch_data import get_competition_id_and_season_id

# seleccionamos el torneo a estudiar (competición, temporada y género)
competition_name = "La Liga"
competition_gender = "male"
season_name = "2015/2016"
season_name_for_filename = "2015_2016"

competition_id, season_id = get_competition_id_and_season_id(competition_name, competition_gender, season_name)
print(f"We are going to study the tournament {competition_name + ' (' + season_name + ', ' + competition_gender + ')'} whose competition_id={competition_id} and season_id={season_id}")

We are going to study the tournament La Liga (2015/2016, male) whose competition_id=11 and season_id=27


Dada esa competición vamos a obtener cuales son todos los partidos que se han jugado en tal competición esa temporada.

In [2]:
from src.fetch_data import get_matches

matches_df = get_matches(competition_id, season_id)
print(f"Number of matches in the competition that season: {matches_df.shape[0]}")

# comprobamos que todos los partidos estén disponibles para obtener información
column_name = "match_status"
if matches_df[column_name].nunique() == 1: 
    print(f"All values in column '{column_name}' are equal: {matches_df[column_name].iloc[0]}")
else:
    print(f"The values in column '{column_name}' are different.")

# ordenamos los partidos por semana de competición y hora de inicio para que un futuro cuando tengamos que buscar datos
# de partidos anteriores podamos ver a partir de que momento consultarlo
matches_sorted_by_week_df = matches_df.sort_values(by=["match_week","kick_off"])

Number of matches in the competition that season: 380
All values in column 'match_status' are equal: available


Tras tener todos los partidos de la competición ordenados a nuestro gusto, nos disponemos a obtener los datos relacionados con eventos, alineaciones... de cada partido. 

Estos datos son ('(x2)' se refiere tanto al equipo de casa como al de fuera y 'x' en el nombre de la variable hace referencia a 'home' y 'away'):
- Estadísticas generales:
  - Tiros:
    - Número de tiros totales (x2) --> total_shots_x
    - Ratio de tiros a puerta (x2) --> shots_on_target_ratio_x
    - Distancia promedio a la portería desde donde se hacen los tiros a puerta (x2) --> average_shot_on_target_distance_x
    - Tiros con alto xG (umbral > 0.2) (x2) --> shots_high_xG_x
    - Número de tiros dentro del área (x2) --> shots_inside_area_x
    - Ratio de tiros dentro del área (x2) --> shots_inside_area_ratio_x
    - Número de tiros con el pie (derecho e izquierdo) (x2) --> shots_foot_x
    - Número de tiros con la cabeza (x2) --> shots_head_x
    - Número de tiros con otra parte del cuerpo (x2) --> shots_other_x
  - Pases:
    - Número de pases realizados (x2) --> total_passes_x
    - Precisión de pases realizados (x2) --> pass_success_ratio_x
    - Número de pases claves realizados (x2) --> key_passes_x
    - Número de pases necesarios para hacer un gol (x2) --> passes_needed_to_make_a_shot_x
    - Número de centros al área ralizados (x2) --> cross_x
    - Precisión de los centros al área (x2) --> cross_success_ratio_x
    - Número de corners (x2) --> corners_x
  - Defensa:
    - Número de intercepciones realizadas con éxito (x2) --> interceptions_won_x
    - Número de recuperaciones/robos (x2) --> recoveries_x
    - Número de bloqueos realizados (x2) --> blocks_x
    - Número de duelos ganados (x2) --> duels_won_x
    - Número de entradas realizadas (x2) --> tackles_x
    - Ratio de entradas exitosas (x2) --> tackles_success_ratio_x
    - Número de faltas cometidas por equipo (x2) --> fouls_committed_x
    - Número de balones divididos (50-50) ganados (x2) --> 50_50_won_x
    - Número de despejes por equipo (x2) --> clearance_x
    - Número de penaltis cometidos (x2) --> penaltys_committed_x
    - Número de errores claves cometidos (x2) --> key_errors_x
    - Número de pérdidas debido a malos controles (x2) --> miscontrol_x
    - Número de tarjetas amarillas (x2) --> yellow_cards_x
    - Número de tarjetas rojas (x2) --> red_cards_x
  - Presión:
    - Número de presiones realizadas (x2) --> pressures_x
    - Número de presiones inmediatas tras pérdida (x2) --> counterpress_x
    - Presiones realizas en tercio ofensivo (x2) --> pressures_in_attacking_third_x
  - Otros:
    - Número de fueras de juego (x2) --> offside_x
    - Número de dribbles intentados (x2) --> dribbles_x
    - Ratio de dribbles exitosos (x2) --> dribbles_success_ratio_x
    - Número de cambios por lesión (x2) --> injuries_substitution_x
    - Número de jugadores lesionados que han abandonado el campo sin hacer una sustitución (x2) --> players_off_x
    - Número de pérdidas de balón (x2) --> dispossessed_x
    - Número de contragolpes/contrataques (x2) --> counterattacks_x
    - Porcentaje de posesión (x2) --> possession_percentage_x
- Estadísticas contextuales:
  - Recuperaciones:
    - Recuperaciones en tercio ofensivo (x2) --> recoveries_attacking_third_x
    - Recuperaciones en tercio medio (x2) --> recoveries_middle_third_x
    - Recuperaciones en tercio defensivo (x2) --> recoveries_defensive_third_x
  - Eventos bajo presión:
    - Tiros realizados bajo presión (x2) --> shots_under_pressure_x
    - Tiros realizados sin presión en el área (x2) --> shots_without_pressure_inside_area_x 
    - Pases realizados bajo presión (x2) --> passes_under_pressure_x
    - Pases realizados sin presión en el área (x2) --> passes_without_pressure_inside_area_x
  - Jugadas a balón parado:
    - Tiros generados desde jugadas a balón parado (x2) --> set_piece_shots_x
    - Tiros en el área generados desde jugadas a balón parado (x2) --> set_piece_shots_inside_area_x
    - Ratio de tiros a puerta generados desde jugadas a balón parado (x2) --> set_piece_shots_on_target_ratio_x
- Tácticas:
- Equipo ganador --> winning_team (home_team, away_team, draw)

In [3]:
from src.preprocessing import process_all_matches
import os


# procesamos todos los partido de la competición
matches_processed_df = process_all_matches(matches_sorted_by_week_df)

# Guardamos la información procesada en un csv por si en algún momento necesitamos volver a tener que cargar esta información
output_dir = "data/processed"
filename = f"{competition_name}({season_name_for_filename}_{competition_gender})_processed.csv"
output_path = os.path.join(output_dir, filename)
matches_processed_df.to_csv(output_path, index=False)
print(f"All matches from the competition processed and save in {output_dir}/{filename}")

All matches from the competition processed and save in data/processed/La Liga(2015_2016_male)_processed.csv


### PRUEBAS

In [4]:
matches_sorted_by_week_df.head(15)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
344,3825565,2015-08-22,18:30:00.000,Spain - La Liga,2015/2016,Espanyol,Getafe,1,0,available,...,None,1,Regular Season,RCDE Stadium,Jesús Gil Manzano,Sergio González Soriano,Francisco Escriba Segura,1.1.0,2,2
345,3825564,2015-08-22,18:30:00.000,Spain - La Liga,2015/2016,RC Deportivo La Coruña,Real Sociedad,0,0,available,...,None,1,Regular Season,Estadio Abanca-Riazor,Antonio Miguel Mateu Lahoz,Víctor Sánchez del Amo,David Moyes,1.1.0,2,2
358,266236,2015-08-23,18:30:00.000,Spain - La Liga,2015/2016,Athletic Club,Barcelona,0,1,available,...,2021-06-13T16:17:31.694,1,Regular Season,San Mamés Barria,NaN,Ernesto Valverde Tejedor,Luis Enrique Martínez García,1.1.0,2,2
193,3825570,2015-08-24,20:30:00.000,Spain - La Liga,2015/2016,Granada,Eibar,1,3,available,...,None,1,Regular Season,Estadio Nuevo Los Cármenes,Ignacio Iglesias Villanueva,José Ramón Sandoval Huertas,José Luis Mendilibar Etxebarria,1.1.0,2,2
194,3825562,2015-08-21,20:30:00.000,Spain - La Liga,2015/2016,Málaga,Sevilla,0,0,available,...,None,1,Regular Season,Estadio La Rosaleda,Alfonso Álvarez Izquierdo,Javier Gracia Carlos,Unai Emery Etxegoien,1.1.0,2,2
342,3825567,2015-08-23,20:30:00.000,Spain - La Liga,2015/2016,Sporting Gijón,Real Madrid,0,0,available,...,None,1,Regular Season,Estadio Municipal El Molinón,Xavier Estrada Fernández,Abelardo Fernández Antuña,Rafael Benítez Maudes,1.1.0,2,2
346,3825563,2015-08-22,20:30:00.000,Spain - La Liga,2015/2016,Atlético Madrid,Las Palmas,1,0,available,...,None,1,Regular Season,Estadio Vicente Calderón,Eduardo Prieto Iglesias,"Tiago Cardoso Mendes, Diego Pablo Simeone",Francisco Herrera Lorenzo,1.1.0,2,2
340,3825569,2015-08-23,22:00:00.000,Spain - La Liga,2015/2016,Levante UD,Celta Vigo,1,2,available,...,None,1,Regular Season,Estadio Ciudad de Valencia,Ricardo De Burgos Bengoetxea,Luis Lucas Alcaraz González,Manuel Eduardo Berizzo,1.1.0,2,2
341,3825568,2015-08-23,22:30:00.000,Spain - La Liga,2015/2016,Real Betis,Villarreal,1,1,available,...,None,1,Regular Season,Estadio Benito Villamarín,Iñaki Bikandi Garrido,José Mel Pérez,Marcelino García Toral,1.1.0,2,2
343,3825566,2015-08-22,22:30:00.000,Spain - La Liga,2015/2016,Rayo Vallecano,Valencia,0,0,available,...,None,1,Regular Season,Estadio de Vallecas,Alberto Undiano Mallenco,Francisco Jémez Martín,Nuno Herlander Simões Espírito Santo,1.1.0,2,2


In [5]:
from statsbombpy import sb
events_df = sb.events(match_id=3825565)
events_df.head()

,ball_receipt_outcome,ball_recovery_recovery_failure,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Espanyol,214,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Getafe,216,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Getafe,216,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:00:00.000,Half Start,NaN


In [6]:
print(events_df['timestamp'].head())

0    00:00:00.000
1    00:00:00.000
2    00:00:00.000
3    00:00:00.000
4    00:00:00.000
Name: timestamp, dtype: object


In [7]:
esta = 'player_off' in events_df.columns
esta

False

In [8]:
events_df['type'].unique()

array(['Starting XI', 'Half Start', 'Pass', 'Ball Receipt*', 'Carry',
       'Pressure', 'Miscontrol', 'Ball Recovery', 'Foul Committed',
       'Foul Won', 'Shot', 'Goal Keeper', 'Clearance', 'Duel',
       'Dribbled Past', 'Dribble', 'Interception', 'Block',
       'Dispossessed', 'Error', 'Shield', 'Half End', 'Substitution',
       'Tactical Shift', 'Injury Stoppage', 'Player Off', 'Player On',
       'Referee Ball-Drop'], dtype=object)

In [9]:
ordenado = events_df.sort_values(by='timestamp', ascending=False)
ordenado

,ball_receipt_outcome,ball_recovery_recovery_failure,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
3279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Getafe,216,00:48:59.610,Half End,NaN
3278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:48:59.610,Half End,NaN
3214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Getafe,216,00:48:42.408,Interception,NaN
1763,Incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:48:42.408,Ball Receipt*,NaN
909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:48:41.739,Pass,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Getafe,216,00:00:00.000,Starting XI,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Getafe,216,00:00:00.000,Half Start,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Espanyol,214,00:00:00.000,Half Start,NaN


In [10]:
ordenado[['minute', 'second', 'timestamp', 'type', 'related_events']]

,minute,second,timestamp,type,related_events
3279,93,59,00:48:59.610,Half End,[4456283a-7c75-4af2-a3be-859646a5eda1]
3278,93,59,00:48:59.610,Half End,[9caa1ff3-b455-4473-996c-1562b8afbfde]
3214,93,42,00:48:42.408,Interception,[984080a2-c6ed-4baa-8546-53c42497d8cc]
1763,93,42,00:48:42.408,Ball Receipt*,[984080a2-c6ed-4baa-8546-53c42497d8cc]
909,93,41,00:48:41.739,Pass,"[47034439-09fb-44e2-8174-c0ffade31cf4, 86d2e20..."
...,...,...,...,...,...
1,0,0,00:00:00.000,Starting XI,NaN
4,45,0,00:00:00.000,Half Start,[ddf8a34e-be7d-4df7-a17e-a4be2b5ea972]
3,0,0,00:00:00.000,Half Start,[3820236b-ffc3-4745-a344-b047cd37a02e]
2,0,0,00:00:00.000,Half Start,[93eba6c1-064b-4508-bb02-cea8720158f2]


In [11]:
hola = 'd6c85afc-f3d3-496d-a081-27dde6cc2ed1' in ordenado['id'].values
hola

False

In [12]:
ordenado['id'].values

array(['9caa1ff3-b455-4473-996c-1562b8afbfde',
       '4456283a-7c75-4af2-a3be-859646a5eda1',
       '47034439-09fb-44e2-8174-c0ffade31cf4', ...,
       '93eba6c1-064b-4508-bb02-cea8720158f2',
       '3820236b-ffc3-4745-a344-b047cd37a02e',
       'f1e3fe8c-cd24-4788-92af-34804d9d0bd5'], dtype=object)

In [13]:
events_df['50_50'].value_counts()

KeyError: '50_50'